# Parts-of-Speech Tagging (POS)

This project we build part-of-speech (POS) tagging for dataset, create and compute transition matrix and transmission matrix in Hidden Markov Models (HMM), implement Viterbi Algorithm through Forward and Backward process (see Coursera). Then finally predict testing word.  

In [1]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np

In [46]:
# Training corpus
with open("WSJ_02-21.pos", 'r') as f:
    training_corpus = f.readlines()
print(f"training corpus")
print(training_corpus[0:10], '\n')

# Test corpus
with open("WSJ_24.pos", 'r') as f:
    y = f.readlines()
print("test corpus")
print(y[0:10], '\n')

with open("hmm_vocab.txt", 'r') as f:
    voc_l = f.read().split('\n')
print("the vocabulary list")
print(voc_l[0:10], '\n')

# Vocabulary dictionary
vocab = {} 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i

training corpus
['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n', 'of\tIN\n', '``\t``\n', 'The\tDT\n', 'Misanthrope\tNN\n', "''\t''\n"] 

test corpus
['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n', 'be\tVB\n', 'taken\tVBN\n', 'from\tIN\n', 'several\tJJ\n', 'vantage\tNN\n'] 

the vocabulary list
['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s"] 



In [15]:
def preprocess(vocab, data_fp):
    orig = []
    prep = []
    with open(data_fp, "r") as data_file:
        for cnt, word in enumerate(data_file):
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue
            # For unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue
            else:
                orig.append(word.strip())
                prep.append(word.strip())
    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))
    return orig, prep

In [23]:
import string
punct = set(string.punctuation)
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]

def assign_unk(tok):
    if any(char.isdigit() for char in tok): # Digits
        return "--unk_digit--"
    elif any(char in punct for char in tok): # Punctuation    
        return "--unk_punct--"    
    elif any(char.isupper() for char in tok): # Upper-case
        return "--unk_upper--"    
    elif any(tok.endswith(suffix) for suffix in noun_suffix): # Nouns
        return "--unk_noun--"    
    elif any(tok.endswith(suffix) for suffix in verb_suffix): # Verbs
        return "--unk_verb--"    
    elif any(tok.endswith(suffix) for suffix in adj_suffix):  # Adjectives
        return "--unk_adj--"    
    elif any(tok.endswith(suffix) for suffix in adv_suffix):  # Adverbs
        return "--unk_adv--"
    return "--unk--"

In [26]:
# corpus without tags, preprocessed
_, prep = preprocess(vocab, "test.words")     

print('The length of the preprocessed test corpus: ', len(prep))
print('This is a sample of the test_corpus: ')
print(prep[0:20])

The length of the preprocessed test corpus:  34199
This is a sample of the test_corpus: 
['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk--', 'points', 'this', 'week', ',', 'with', 'readings', 'on', 'trade', ',', 'output']


`emission_counts`: maps (tag, word) to the number of times it happened. 
`transition_counts`: maps (prev_tag, tag) to the number of times it has appeared. 
`tag_counts`: maps (tag) to the number of times it has occured. 

In [30]:
def get_word_tag(line, vocab): 
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word, tag = line.split()
        if word not in vocab: 
            # Handle unknown words
            word = assign_unk(word)
        return word, tag
    return None 

def create_dictionaries(training_corpus, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)    
    prev_tag = '--s--' 
    
    i = 0     
    for word_tag in training_corpus:
        i += 1       
        word, tag = get_word_tag(word_tag,vocab)        
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag        
    return emission_counts, transition_counts, tag_counts

emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

In [48]:
# get all the POS states
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 46
View these POS tags (states)
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [55]:
print("transition examples: ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print("ambiguous word example: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'back': print (tup, cnt) 

transition examples: 
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)
emission examples: 
(('DT', 'any'), 721)
(('NN', 'decrease'), 7)
(('NN', 'insider-trading'), 5)
ambiguous word example: 
('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


In [34]:
def predict_pos(prep, y, emission_counts, vocab, states):
    num_correct = 0
    all_words = set(emission_counts.keys())
    total = len(y)
    for word, y_tup in zip(prep, y): 
        y_tup_l = y_tup.split()
        if len(y_tup_l) == 2:
            true_label = y_tup_l[1]
        else:
            continue    
        count_final = 0
        pos_final = ''

        if word in vocab:
            for pos in states:
                key = (pos,word)
                if key in emission_counts: # complete this line
                    count = emission_counts[key]
                    if count>count_final: # complete this line
                        count_final = count
                        pos_final = pos
            if pos_final == true_label:
                num_correct += 1            
    accuracy = num_correct / total    
    return accuracy

accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.8889


A Markov Model utilizes a transition  matrix,  `A`.    
A Hidden Markov Model adds an observation or emission matrix `B` when we are in a particular state. 

In [49]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)    
    A = np.zeros((num_tags,num_tags))
    trans_keys = set(transition_counts.keys())
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            key = (all_tags[i],all_tags[j])
            if key in transition_counts: #complete this line
                count = transition_counts[key]
            count_prev_tag = tag_counts[all_tags[i]]
            A[i,j] = (count + alpha) / (count_prev_tag + alpha*num_tags)   
    return A

In [50]:
alpha = 0.001
for i in range(46):
    tag_counts.pop(i,None)
A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

#print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691
              RBS            RP           SYM        TO            UH
RBS  2.217069e-06  2.217069e-06  2.217069e-06  0.008870  2.217069e-06
RP   3.756509e-07  7.516775e-04  3.756509e-07  0.051089  3.756509e-07
SYM  1.722772e-05  1.722772e-05  1.722772e-05  0.000017  1.722772e-05
TO   4.477336e-05  4.472863e-08  4.472863e-08  0.000090  4.477336e-05
UH   1.030439e-05  1.030439e-05  1.030439e-05  0.061837  3.092348e-02


In [37]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    num_tags = len(tag_counts)
    all_tags = sorted(tag_counts.keys())
    num_words = len(vocab)
    B = np.zeros((num_tags, num_words))
    emis_keys = set(list(emission_counts.keys()))
    for i in range(num_tags): 
        for j in range(num_words): 
            count = 0
            key = (all_tags[i],vocab[j])
            if key in emission_counts.keys(): 
                count = emission_counts[key]
            count_tag = tag_counts[all_tags[i]]
            B[i,j] = (count + alpha) / (count_tag+ alpha*num_words)
    return B

In [56]:
for i in range(46\):
    tag_counts.pop(i,None)

B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))
print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")

cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']
cols = [vocab[a] for a in cidx]
rvals =['CD','NN','NNS', 'VB','RB','RP']
rows = [states.index(a) for a in rvals]
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


In [41]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    num_tags = len(tag_counts)
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    s_idx = states.index("--s--")
    for i in range(num_tags):         
        if A[s_idx,i] == 0: 
            best_probs[i,0] = float('-inf')
        else:
            best_probs[i,0] = math.log(A[s_idx,i]) + math.log(B[i,vocab[corpus[0]]] )
    return best_probs, best_paths

best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -22.6098
best_paths[2,3]: 0.0000


In [43]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    num_tags = best_probs.shape[0]
    for i in range(1, len(test_corpus)): 
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))
        for j in range(num_tags): # complete this line
            best_prob_i =  float("-inf")
            best_path_i = None
            for k in range(num_tags): # complete this line
                prob = best_probs[k,i-1]+math.log(A[k,j]) +math.log(B[j,vocab[test_corpus[i]]])
                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
            best_probs[j,i] = best_prob_i
            best_paths[j,i] = best_path_i
    return best_probs, best_paths

best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)
# Test this function 
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}") 

Words processed:     5000
Words processed:    10000
Words processed:    15000
Words processed:    20000
Words processed:    25000
Words processed:    30000
best_probs[0,1]: -24.7822
best_probs[0,4]: -49.5601


In [44]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    m = best_paths.shape[1] 
    z = [None] * m
    num_tags = best_probs.shape[0]
    best_prob_for_last_word = float('-inf')

    pred = [None] * m
    for k in range(num_tags): # complete this line
        if best_probs[k, m - 1]>best_prob_for_last_word: 
            best_prob_for_last_word = best_probs[k, m - 1]
            z[m - 1]=k
    pred[m - 1] = states[z[m - 1]]

    for i in range(m-1, -1, -1): # complete this line
        pos_tag_for_word_i = z[i]
        z[i - 1] = best_paths[pos_tag_for_word_i,i]
        pred[i - 1] = states[z[i - 1]]
    return pred

pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

# Predicting on a data set
print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']
The third word is: temperature
Your prediction is: NN
Your corresponding label y is:  temperature	NN



In [52]:
def compute_accuracy(pred, y):
    num_correct = 0
    total = 0
    for prediction, y in zip(pred, y):
        word_tag_tuple = y.split()
        if len(word_tag_tuple)!=2:
            continue 
        word, tag = word_tag_tuple
        if prediction == tag: # complete this line
            num_correct += 1
        total += 1
    return (num_correct/total)
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.9531


### References
- https://www.coursera.org/learn/probabilistic-models-in-nlp?
- ["Speech and Language Processing", Dan Jurafsky and James H. Martin](https://web.stanford.edu/~jurafsky/slp3/)
